# Analisis de datos del COVID-19

Objetivos del análisis:
* Automatizar descarga y tratamiento de datos.
* Generar salidas reutilizables.
* Generar gráficos explicativos, a poder ser múltiples.
* Separar los datos en dos tablas y hacer el tratamiento eficiente completo.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
dfbase = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv')

In [14]:
df=dfbase

In [15]:
df.columns

Index(['dateRep', 'year_week', 'cases_weekly', 'deaths_weekly',
       'countriesAndTerritories', 'geoId', 'countryterritoryCode',
       'popData2019', 'continentExp',
       'notification_rate_per_100000_population_14-days'],
      dtype='object')

In [16]:
df.shape

(8938, 10)

In [17]:
df['dateRep'] = pd.to_datetime(df['dateRep'], format='%d/%m/%Y', errors='ignore')

In [18]:
df = df.drop(['day','month','year','geoId', 'countryterritoryCode', 
              'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'], axis = 1)

KeyError: "['day' 'month' 'year'\n 'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'] not found in axis"

In [ ]:
renombrados={'dateRep':'Fecha', 'cases':'Casos', 'deaths':'Muertes',
                   'countriesAndTerritories':'País','continentExp':'Continente','popData2019':'Población'}
df.rename(columns=renombrados, inplace=True)

In [ ]:
df = df.set_index(['País', 'Fecha']).sort_index()

In [ ]:
df['Acumulado Casos'] = df.groupby(['País'])['Casos'].cumsum()

Faltan las medias moviles

In [ ]:
df['Acumulado Muertes'] = df.groupby(['País'])['Muertes'].cumsum()

In [ ]:
df['Ratio'] = (df['Acumulado Muertes'] / df['Acumulado Casos']).fillna(0)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df=df[['Continente', 'País', 'Fecha', 'Casos','Acumulado Casos', 'Muertes', 'Acumulado Muertes','Ratio', 'Población']]

In [ ]:
df.info()

In [ ]:
sns.countplot(y='Continente', data=df)

In [ ]:
df.Casos.sum()

In [ ]:
agrupadocontinente=df.groupby(['Continente'])['Casos'].sum().reset_index().sort_values('Casos', ascending=False)
agrupadocontinente.plot.bar(x='Continente', y='Casos')

In [ ]:
agrupadopais=df.groupby(['País', 'Población'])['Casos'].sum().reset_index()
# agrupadopais.head()[df['País'] == 'Spain']
agrupadopais

In [ ]:
sns.regplot(x='Población', y ='Casos', data=agrupadopais)

Implementar el encadenado de metodos.

In [ ]:
df2 = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv')
df2['dateRep'] = pd.to_datetime(df2['dateRep'], format='%d/%m/%Y', errors='ignore')
df2 = df2.drop(['day','month','year','geoId', 'countryterritoryCode', 'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'], axis = 1)
df2 = df2.set_index(['countriesAndTerritories', 'dateRep']).sort_index()
df2['cases acumulated'] = df2.groupby(['countriesAndTerritories'])['cases'].cumsum()
df2['deaths acumulated'] = df2.groupby(['countriesAndTerritories'])['deaths'].cumsum()
df2.index.names = ['País', 'Fecha']
df2.columns = ['Casos','Muertes','Población', 'Continente', 'Acumulado Casos', 'Acumulado Muertes']
df2.reset_index(inplace=True)b

Separar en grupos y hacer un merge para fusionar los datos.

## Method Chaining



In [ ]:
salidacovid = (covid
               .drop(['day', 'month', 'year', 'geoId', 'countryterritoryCode',
                      'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'], axis=1)
               .rename(columns={'dateRep': 'Fecha', 'cases': 'Casos',
                                'deaths': 'Muertes', 'countriesAndTerritories': 'País',
                                'popData2019': 'Población', 'continentExp': 'Continente'})
               .assign(Fecha=lambda x: pd.to_datetime(x['Fecha'],  format='%d/%m/%Y'))
               .sort_values(by=['País', 'Fecha'], ascending=[True, True])
               .assign(AcumuladoCasos=lambda x: x.groupby('País')['Casos'].transform('cumsum'),
                       AcumuladoCasos14=lambda x: 100000 * x.groupby('País')['Casos'].transform(
                   lambda x: x.rolling(14).sum()) / x['Población'],
                   AcumuladoMuertes=lambda x: x.groupby(
                   'País')['Muertes'].transform('cumsum'),
                   PorcentajeAcumuladoCasos=lambda x: x['AcumuladoCasos'] /
                   x['Población'],
                   PorcentajeAcumuladoMuertes=lambda x: x['AcumuladoMuertes'] / x['Población'])
               .set_index(['País', 'Fecha'])
               .sort_index()
               )
# paises = ['Spain']
# salidacovid[salidacovid['País'] == 'Spain']['AcumuladoCasos14'].plot()
# salidacovid[salidacovid['País'] == 'Germany']['AcumuladoCasos14'].plot()

In [ ]:
salidacovid.loc[['Spain', 'Germany']]['AcumuladoCasos14'].unstack(level=0).plot();